<pre>
Copyright 2021-2022 Boris Shminke

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
</pre>

In [1]:
# first, we should have a file with a TPTP problem
import sys
import os
if sys.version_info.major == 3 and sys.version_info.minor >= 9:
    from importlib.resources import files
else:
    from importlib_resources import files

syllogism = files("gym_saturation").joinpath(
    os.path.join(
        "resources", "TPTP-mock", "Problems", "TST", "TST003-1.p"
    )
)

In [2]:
# every carnivore is an animal
# jaguar is a carnivore
# thus, jaguar is an animal
!cat {syllogism}

cnf(p_imp_q, hypothesis, ~ carnivore(X) | animal(X)).
cnf(p, hypothesis, carnivore(jaguar)).
cnf(q, hypothesis, ~ animal(jaguar)).


/home/boris/projects/gym-saturation/venv/lib/python3.10/site-packages/jupyter_client/jsonutil.py:76: DeprecationWarning: Interpreting naive datetime as local 2022-10-14 14:47:32. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [3]:
# we have a vampire installed in this container
!vampire --version

Vampire 4.6.1 (commit b6b253a1c on 2021-10-18 21:09:27 +0200)
Linked with Z3 4.8.13.0 f03d756e086f81f2596157241e0decfb1c982299 z3-4.8.4-5390-gf03d756e0


In [4]:
# then we create and reset a OpenAI Gym environment
import gym

env = gym.make(
    "GymVampire-v0",
    # a classical syllogism
    problem_list=[syllogism],
    vampire_binary_path="vampire"
)
env.reset()
# we can render the environment in a human readable form
print("starting hypotheses:")
print(env.render(mode="human"))

starting hypotheses:
cnf(1, lemma, animal(X0) | ~carnivore(X0), inference(input, [], [])).
cnf(2, lemma, carnivore(jaguar), inference(input, [], [])).
cnf(3, lemma, ~animal(jaguar), inference(input, [], [])).


/home/boris/projects/gym-saturation/venv/lib/python3.10/site-packages/jupyter_client/jsonutil.py:76: DeprecationWarning: Interpreting naive datetime as local 2022-10-14 14:47:34. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [5]:
# our 'age' agent will always select clauses for inference
# in the order they appeared in current proof attempt
action = 0
done = False
while not done:
    observation, reward, done, info = env.step(action)
    action += 1

/home/boris/projects/gym-saturation/venv/lib/python3.10/site-packages/jupyter_client/jsonutil.py:76: DeprecationWarning: Interpreting naive datetime as local 2022-10-14 14:47:35. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [6]:
# SaturationEnv has an additional method
# for extracting only clauses which became parts of the proof
# (some steps might be unnecessary to find the proof)
print("refutation proof:")
print(env.tstp_proof)
print(f"number of attempted steps: {action}")

refutation proof:
cnf(1, lemma, animal(X0) | ~carnivore(X0), inference(input, [], [])).
cnf(2, lemma, carnivore(jaguar), inference(input, [], [])).
cnf(3, lemma, ~animal(jaguar), inference(input, [], [])).
cnf(4, lemma, animal(jaguar), inference(resolution, [], [2, 1])).
cnf(5, lemma, $false, inference(subsumption_resolution, [], [4, 3])).
number of attempted steps: 2


/home/boris/projects/gym-saturation/venv/lib/python3.10/site-packages/jupyter_client/jsonutil.py:76: DeprecationWarning: Interpreting naive datetime as local 2022-10-14 14:47:36. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)
